In [45]:
import cv2

import numpy as np
def preprocess_image(image):
    # Görüntüyü gri tonlamaya çevir
    #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Gaussian Blur uygula
    blurred_image = cv2.medianBlur(image, ksize=5)
    
    blurred_image = cv2.resize(blurred_image, (64, 64))
    
    return blurred_image

In [2]:
def extract_hog_features(image):
    winSize = (64,64)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)
    h = hog.compute(image)
    return h.flatten()

In [50]:
def calculate_geometric_features(landmarks):
    features = []
    if landmarks is not None:
        # Gözler arası mesafe
        eye_distance = np.linalg.norm(landmarks[36] - landmarks[45])
        features.append(eye_distance)
        # Burun uzunluğu
        nose_length = np.linalg.norm(landmarks[27] - landmarks[33])
        features.append(nose_length)
        # Ağız genişliği
        mouth_width = np.linalg.norm(landmarks[48] - landmarks[54])
        features.append(mouth_width)
        # Diğer özellikler...
    return features

In [47]:
from skimage.color import rgb2gray
from sklearn.preprocessing import normalize
import numpy as np

def extract_lbp_features(image, P=8, R=1):
    # Görüntüyü gri tonlamaya çevir
    image_gray = rgb2gray(image)
    # LBP uygula
    lbp = local_binary_pattern(image_gray, P, R, method="uniform")
    # Histogramı hesapla ve normalize et
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

In [52]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import random
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


def read_and_split_data(directory_path, test_size=0.2):
    images = []
    labels = []
    label_names = []
    combined_features_list = []
    random.seed(43)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for root, dirs, files in os.walk(directory_path):
        for dir in dirs:
            current_dir_path = os.path.join(root, dir)
            image_files = [f for f in os.listdir(current_dir_path) if f.endswith(".jpg") or f.endswith(".png")]
            # Select a random subset of images if needed
            #selected_files = random.sample(image_files, min(len(image_files), 100))
            
            for file in image_files:
                image_path = os.path.join(current_dir_path, file)
                image = cv2.imread(image_path)
                
                if image is not None:
                    faces = face_cascade.detectMultiScale(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 1.1, 4)
                    for (x, y, w, h) in faces:
                        face_img = image[y:y+h, x:x+w]
                        face_img = preprocess_image(face_img)
                        hog_features = calculate_geometric_features(face_img)
                        combined_features_list.append(hog_features)
                        labels.append(dir)
            if dir not in label_names:
                label_names.append(dir)

    
    
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(combined_features_list, labels, test_size=test_size, stratify=labels)
    return X_train, X_test, y_train, y_test

In [49]:
def read_and_split_data_lpb(directory_path, test_size=0.2):
    combined_features_list = []
    labels = []
    label_names = []
    random.seed(43)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for root, dirs, files in os.walk(directory_path):
        for dir in dirs:
            current_dir_path = os.path.join(root, dir)
            image_files = [f for f in os.listdir(current_dir_path) if f.endswith(".jpg") or f.endswith(".png")]
            
            for file in image_files:
                image_path = os.path.join(current_dir_path, file)
                image = cv2.imread(image_path)
                
                if image is not None:
                    faces = face_cascade.detectMultiScale(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 1.1, 4)
                    for (x, y, w, h) in faces:
                        face_img = image[y:y+h, x:x+w]
                        face_img = preprocess_image(face_img)  # preprocess_image fonksiyonunun tanımını sağlamalısınız.
                        lbp_features = extract_lbp_features(face_img)
                        combined_features_list.append(lbp_features)
                        labels.append(dir)
            if dir not in label_names:
                label_names.append(dir)
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(combined_features_list, labels, test_size=test_size, stratify=labels)
    return X_train, X_test, y_train, y_test

In [40]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
def calculate_accuracy(X_train, y_train, X_test, y_test):
    lda = LDA(n_components=min(len(np.unique(y_train))-1, len(X_train[0])))
    X_train_lda = lda.fit_transform(X_train, y_train)
    X_test_lda = lda.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=30)
    knn.fit(X_train_lda, y_train)
    y_pred = knn.predict(X_test_lda)
    accuracy = accuracy_score(y_test, y_pred)

    best_svm = SVC(C=1000.0, class_weight='balanced', gamma=0.01)
    best_svm.fit(X_train_lda, y_train)

    # Make predictions with the best SVM
    y_pred_svm = best_svm.predict(X_test_lda)

    # Calculate and return the accuracy for SVM
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    return accuracy,accuracy_svm

In [51]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def calculate_accuracy_with_svm(X_train, y_train, X_test, y_test):
    # LDA nesnesini başlat ve n_components ayarla
    lda = LDA(n_components=min(len(np.unique(y_train))-1, len(X_train[0])))
    
    # Eğitim verisi üzerinde LDA'yı fit edin ve dönüştürme uygulayın
    X_train_lda = lda.fit_transform(X_train, y_train)
    
    # Test verisini LDA ile dönüştürün
    X_test_lda = lda.transform(X_test)
    
    # SVM sınıflandırıcısını başlat
    svm = SVC(kernel='linear')  # Kernel tipi olarak 'linear' kullanıldı, ihtiyaca göre 'rbf' veya başka bir kernel seçilebilir
    
    # Eğitim verisi üzerinde SVM'yi fit edin
    svm.fit(X_train_lda, y_train)
    
    # Test verisi üzerinde tahmin yap
    y_pred = svm.predict(X_test_lda)
    
    # Doğruluk oranını hesapla
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [32]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
def calculate_accuracy_with_pca(X_train, y_train, X_test, y_test):
    # PCA nesnesini başlatın. n_components, azaltılmış özellik uzayının boyutunu belirler.
    # Örneğin, veri setinizin varyansının büyük bir kısmını korumak için 0.95 gibi bir değer kullanabilirsiniz.
    lda = LDA(n_components=min(len(np.unique(y_train))-1, len(X_train[1])))
    X_train_lda = lda.fit_transform(X_train, y_train)
    X_test_lda = lda.transform(X_test)

    X_train = np.array(X_train_lda)
    X_test = np.array(X_test_lda)
    #pca = PCA(n_components=105)
    
    pca = PCA(n_components=104, svd_solver='randomized', whiten=True).fit(X_train)
    # Eğitim verisi üzerinde PCA'yı fit edin ve dönüşümü uygulayın
    X_train_pca = pca.fit_transform(X_train)

    # Test verisini dönüştürün
    X_test_pca = pca.transform(X_test)

    # KNN sınıflandırıcısını başlatın
    knn = KNeighborsClassifier(n_neighbors=30)

    # Eğitim verisi üzerinde KNN'yi fit edin
    knn.fit(X_train_pca, y_train)

    # Test verisi üzerinde tahmin yapın
    y_pred = knn.predict(X_test_pca)

    # Doğruluk oranını hesaplayın
    accuracy = accuracy_score(y_test, y_pred)

    best_svm = SVC(C=1000.0, class_weight='balanced', gamma=0.01)
    best_svm.fit(X_train_pca, y_train)

    # Make predictions with the best SVM
    y_pred_svm = best_svm.predict(X_test_pca)

    # Calculate and return the accuracy for SVM
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    return accuracy, accuracy_svm

In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def calculate_accuracy_with_rf(X_train, y_train, X_test, y_test):
    # LDA ile boyut indirgeme
    lda = LDA(n_components=min(len(np.unique(y_train))-1, len(X_train[0])))
    X_train_lda = lda.fit_transform(X_train, y_train)
    X_test_lda = lda.transform(X_test)

    # Random Forest sınıflandırıcısını başlat
    rf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators, ormanınızdaki ağaç sayısıdır

    # Eğitim verisi üzerinde Random Forest'ı eğit
    rf.fit(X_train_lda, y_train)

    # Test verisi üzerinde tahmin yap
    y_pred = rf.predict(X_test_lda)

    # Doğruluk oranını hesapla
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [53]:
X_train, X_test, y_train, y_test = read_and_split_data("105_classes_pins_dataset")


AttributeError: 'list' object has no attribute 'flatten'

In [49]:
accuracy,accuracy_svm = calculate_accuracy(X_train, y_train, X_test, y_test)
print("accuracy Etiket:", accuracy, "With SVM: ",accuracy_svm)

accuracy Etiket: 0.02254684026675135 With SVM:  0.0219117180057161
